### Exploratory Data Analysis (EDA) for Histopathology Cancer Detection
This notebook is dedicated to analyzing and understanding the histopathology cancer detection dataset. We'll explore data distributions, check for any patterns, and perform preliminary visualizations. Additionally, we’ll use tools like Pandas Profiling for a quick overview of dataset characteristics.

#### Import Libraries

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv
from PIL import Image

from scripts.config import LABELS_FILE, TRAIN_DIR, TARGET_SIZE
from scripts.data_utils import HistologyDataset, display_sample_images
from scripts.preprocessing import Preprocessing

#### Load Data

In [ ]:
# Load labels from the training labels CSV
labels_df = pd.read_csv(LABELS_FILE)
print("Data Loaded Successfully")
print(labels_df.head())

# Initialize preprocessing pipeline with automatic reference selection
preprocessor = Preprocessing(
    input_size=TARGET_SIZE,
    stain_correction=True,
    stain_method="macenko",
    mask_size=(32, 32),
    img_dir=TRAIN_DIR  # Automatically select reference image from this directory
)
print(f"Selected reference image: {preprocessor.stain_reference_image_path}")

In [ ]:
# Display the selected reference image
reference_image = Image.open(preprocessor.stain_reference_image_path)
plt.imshow(reference_image)
plt.title("Selected Stain Reference Image")
plt.axis("off")
plt.show()

In [ ]:
# Initialize train_dataset
train_dataset = HistologyDataset(
    dataframe=labels_df,
    img_dir=TRAIN_DIR,
    preprocess_pipeline=preprocessor,
    mode="train"
)

#### Basic Information and Summary Statistics

In [ ]:
# Display basic info
print("Dataset Info:")
labels_df.info()

print("\nSummary Statistics:")
labels_df.describe()

#### Class Distribution

In [ ]:
# Visualize class distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='label', data=labels_df, palette='pastel')
plt.title('Class Distribution of Cancer Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

#### Sample Images

In [ ]:
# Display preprocessed samples for label 0 (non-cancerous)
print("Preprocessed Non-cancerous samples (Label 0):")
display_sample_images(
    dataset=train_dataset,
    label=0,
    sample_size=5
)

In [ ]:
# Display preprocessed samples for label 1 (cancerous)
print("Preprocessed Cancerous samples (Label 1):")
display_sample_images(
    dataset=train_dataset,
    label=1,
    sample_size=5
)

#### SweetViz Report

In [ ]:
# Generate an automated EDA report using Sweetviz
eda_report = sv.analyze(labels_df)
eda_report.show_html(filepath='eda_report.html', open_browser=False)
print("Sweetviz Report Generated: eda_report.html")

#### Conclusion
This concludes the exploratory data analysis:
- The dataset's class distribution is imbalanced, favoring non-cancerous images.
- Sweetviz provides a comprehensive report for further insights.
- Images are uniform in size but will be resized to the target dimensions during preprocessing.
